In [1]:
import pandas as pd
import requests

In [2]:
input_dir = "/Users/guangrong/Documents/GitHub_project/fastqpi_BigGIM/KGs/TCGA_mut/"

In [3]:
# Mondo reference: https://raw.githubusercontent.com/monarch-initiative/mondo/master/reports/release/semantic-xref-pairs.tsv"

In [4]:
#generating a ID mapping dictionary for disease name and mondo ID
TCGA_Mondo_map = pd.read_csv(input_dir + "Mondo_TCGA.csv")
TCGA_Mondo_map
dic_mondo_map = {}
for i in range(0, TCGA_Mondo_map.shape[0]):
    dic_mondo_map['TCGA_' + list(TCGA_Mondo_map['Study Abbreviation'])[i]] = list(TCGA_Mondo_map['MONDO'])[i]
    dic_mondo_map['TCGA-' + list(TCGA_Mondo_map['Study Abbreviation'])[i]] = list(TCGA_Mondo_map['MONDO'])[i]


In [5]:
#Generate an ID mapping dictionary for gene symbol and NCBI gene ID
ID_map = pd.read_csv("/Users/guangrong/Documents/GitHub_project/fastqpi_BigGIM/KGs/ID_convert/Table_ID_convert_9.19.2021.tsv", sep = '\t')
dic_ID_symbol2ncbi ={}
dic_ID_ncbi2symbol = {}
import math
ID_map.astype(str)
for i in range(0,ID_map.shape[0]):
    symbol = ID_map['Approved_symbol'][i]
    
    ncbi_id = ID_map['NCBI_Gene_ID'][i]
    ensemble_id = ID_map['Ensembl_gene_ID'][i]
    if math.isnan(ncbi_id) == False :
        ncbi_id = int(ncbi_id)
        if symbol not in dic_ID_symbol2ncbi:
            dic_ID_symbol2ncbi[symbol] = ncbi_id
        else:
            print(symbol)
for i in range(0,ID_map.shape[0]):
    alias_list = str(ID_map["Alias_symbols"][i]).split(',')
    ncbi_id = ID_map['NCBI_Gene_ID'][i]
    ensemble_id = ID_map['Ensembl_gene_ID'][i]
    if len(alias_list) > 0:
        for alias in alias_list:
            if alias not in dic_ID_symbol2ncbi:
                dic_ID_symbol2ncbi[alias] = ncbi_id
dic_ID_symbol2ncbi['MEN1'] = 4221
dic_ID_symbol2ncbi['TSC1']= 7248
dic_ID_symbol2ncbi['WHSC1'] = 7468
dic_ID_symbol2ncbi['MET'] = 4233
dic_ID_symbol2ncbi['HIST1H1E'] = 3008
dic_ID_symbol2ncbi['FLNA'] = 2316
dic_ID_symbol2ncbi["FAM46D"] = 169966
dic_ID_symbol2ncbi["HIST1H1C"] = 3006

In [6]:
input_dir = "/Users/guangrong/Documents/GitHub_project/fastqpi_BigGIM/KGs/TCGA_mut/"

In [7]:
# Load TCGA_mutation_frequency data and filtering
df = pd.read_excel(input_dir+"TCGA_Mut_Freq.xlsx")
df_high_freq = df.loc[df['frequency_qualifier'] > 0.05]
df_high_freq = df_high_freq.loc[df_high_freq['has_count'] >= 5]

In [8]:
# Filter genes from the literature reported driver genes
cancer_drivers = pd.read_csv(input_dir + "cancer_drivers_PMID_29625053.csv") # supplemental tablefrom PMID_29625053
driver_tcga_freq =  df_high_freq.loc[df_high_freq['subject_name'].isin(cancer_drivers['Gene'])]

In [9]:
# Formating columns

In [10]:
driver_tcga_freq['subject_category'] = ['Gene'] * driver_tcga_freq.shape[0]

/var/folders/w1/p1v0w3j165ggfdk4xpn0tzgh0000gp/T/ipykernel_84617/23269610.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driver_tcga_freq['subject_category'] = ['Gene'] * driver_tcga_freq.shape[0]


In [11]:
driver_tcga_freq['subject_id_prefix'] = ['NCBIGene'] * driver_tcga_freq.shape[0]

/var/folders/w1/p1v0w3j165ggfdk4xpn0tzgh0000gp/T/ipykernel_84617/3421882735.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driver_tcga_freq['subject_id_prefix'] = ['NCBIGene'] * driver_tcga_freq.shape[0]


In [12]:
driver_tcga_freq['object_category'] = "Disease"

/var/folders/w1/p1v0w3j165ggfdk4xpn0tzgh0000gp/T/ipykernel_84617/4081730457.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driver_tcga_freq['object_category'] = "Disease"


In [13]:
driver_tcga_freq['object_id_prefix'] = "MONDO"

/var/folders/w1/p1v0w3j165ggfdk4xpn0tzgh0000gp/T/ipykernel_84617/2049567035.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driver_tcga_freq['object_id_prefix'] = "MONDO"


In [14]:
driver_tcga_freq['predicate'] = "gene_associated_with_condition"

/var/folders/w1/p1v0w3j165ggfdk4xpn0tzgh0000gp/T/ipykernel_84617/4143724263.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driver_tcga_freq['predicate'] = "gene_associated_with_condition"


In [15]:
driver_tcga_freq['subject_aspect_qualifier'] = "Mutation"

/var/folders/w1/p1v0w3j165ggfdk4xpn0tzgh0000gp/T/ipykernel_84617/76024736.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driver_tcga_freq['subject_aspect_qualifier'] = "Mutation"


In [16]:
driver_tcga_freq['supporting_study_cohort'] = "TCGA"

/var/folders/w1/p1v0w3j165ggfdk4xpn0tzgh0000gp/T/ipykernel_84617/2592182837.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driver_tcga_freq['supporting_study_cohort'] = "TCGA"


In [17]:
object_id_list = []
for i in range(0,driver_tcga_freq.shape[0] ):
    object_id_list.append(dic_mondo_map[list(driver_tcga_freq["object_name"])[i]])
    
driver_tcga_freq['object_id'] = object_id_list

/var/folders/w1/p1v0w3j165ggfdk4xpn0tzgh0000gp/T/ipykernel_84617/2287823909.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driver_tcga_freq['object_id'] = object_id_list


In [18]:
subject_id_list = []
for i in range(0,driver_tcga_freq.shape[0]):
    
    subject_id_list.append(dic_ID_symbol2ncbi[list(driver_tcga_freq['subject_name'])[i]])

driver_tcga_freq['subject_id'] = subject_id_list

/var/folders/w1/p1v0w3j165ggfdk4xpn0tzgh0000gp/T/ipykernel_84617/2374070043.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driver_tcga_freq['subject_id'] = subject_id_list


In [19]:
driver_tcga_freq

,subject_name,subject_id,subject_category,subject_id_prefix,object_name,object_id,object_category,object_id_prefix,predicate,frequency_qualifier,subject_aspect_qualifier,has_count,supporting_study_cohort
0,TP53,7157,Gene,NCBIGene,TCGA-ACC,MONDO:0008734,Disease,MONDO,gene_associated_with_condition,0.184783,Mutation,17,TCGA
2,CTNNB1,1499,Gene,NCBIGene,TCGA-ACC,MONDO:0008734,Disease,MONDO,gene_associated_with_condition,0.152174,Mutation,14,TCGA
10,MEN1,4221,Gene,NCBIGene,TCGA-ACC,MONDO:0008734,Disease,MONDO,gene_associated_with_condition,0.076087,Mutation,7,TCGA
18,APOB,338,Gene,NCBIGene,TCGA-ACC,MONDO:0008734,Disease,MONDO,gene_associated_with_condition,0.065217,Mutation,6,TCGA
23,PRKAR1A,5573,Gene,NCBIGene,TCGA-ACC,MONDO:0008734,Disease,MONDO,gene_associated_with_condition,0.054348,Mutation,5,TCGA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
331223,GNAQ,2776,Gene,NCBIGene,TCGA-UVM,MONDO:0006486,Disease,MONDO,gene_associated_with_condition,0.500000,Mutation,40,TCGA
331224,GNA11,2767,Gene,NCBIGene,TCGA-UVM,MONDO:0006486,Disease,MONDO,gene_associated_with_condition,0.450000,Mutation,36,TCGA
331225,SF3B1,23451,Gene,NCBIGene,TCGA-UVM,MONDO:0006486,Disease,MONDO,gene_associated_with_condition,0.225000,Mutation,18,TCGA
331226,BAP1,8314,Gene,NCBIGene,TCGA-UVM,MONDO:0006486,Disease,MONDO,gene_associated_with_condition,0.162500,Mutation,13,TCGA


In [20]:
driver_tcga_freq.to_csv("../Input_Table/TCGA_driver_mut_freq.csv", index = None)

In [21]:
driver_tcga_freq.loc[driver_tcga_freq['subject_name'] == 'FLT3']

,subject_name,subject_id,subject_category,subject_id_prefix,object_name,object_id,object_category,object_id_prefix,predicate,frequency_qualifier,subject_aspect_qualifier,has_count,supporting_study_cohort
141061,FLT3,2322,Gene,NCBIGene,TCGA_LAML,MONDO:0018874,Disease,MONDO,gene_associated_with_condition,0.285714,Mutation,56,TCGA
262264,FLT3,2322,Gene,NCBIGene,TCGA-SKCM,MONDO:0005012,Disease,MONDO,gene_associated_with_condition,0.105150,Mutation,49,TCGA
308233,FLT3,2322,Gene,NCBIGene,TCGA-UCEC,MONDO:0000553,Disease,MONDO,gene_associated_with_condition,0.083019,Mutation,44,TCGA


In [22]:
driver_tcga_freq.columns

Index(['subject_name', 'subject_id', 'subject_category', 'subject_id_prefix',
       'object_name', 'object_id', 'object_category', 'object_id_prefix',
       'predicate', 'frequency_qualifier', 'subject_aspect_qualifier',
       'has_count', 'supporting_study_cohort'],
      dtype='object')

In [23]:
#End